## Deliverable 2. Create a Customer Travel Destinations Map.

In [144]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [145]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..","Weather_Database","WeatherPy_database.csv"))
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,50.18,73,100,4.63,overcast clouds
1,1,Hilo,US,19.7297,-155.0900,69.53,86,100,5.75,light rain
2,2,Hobart,AU,-42.8794,147.3294,55.26,86,20,5.75,few clouds
3,3,Hokitika,NZ,-42.7167,170.9667,52.92,82,38,6.06,scattered clouds
4,4,Lagoa,PT,39.0500,-27.9833,60.58,83,68,18.39,broken clouds
5,5,Aksu,CN,41.1231,80.2644,76.42,13,11,5.37,few clouds
6,6,Sassandra,CI,4.9500,-6.0833,87.64,63,93,9.78,overcast clouds
7,7,Hithadhoo,MV,-0.6000,73.0833,83.62,64,57,7.94,broken clouds
8,8,Klaksvik,FO,62.2266,-6.5890,36.52,74,98,27.34,overcast clouds
9,9,Avarua,CK,-21.2078,-159.7750,80.65,83,40,5.75,scattered clouds


In [146]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [147]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & 
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hilo,US,19.7297,-155.0900,69.53,86,100,5.75,light rain
5,5,Aksu,CN,41.1231,80.2644,76.42,13,11,5.37,few clouds
6,6,Sassandra,CI,4.9500,-6.0833,87.64,63,93,9.78,overcast clouds
7,7,Hithadhoo,MV,-0.6000,73.0833,83.62,64,57,7.94,broken clouds
9,9,Avarua,CK,-21.2078,-159.7750,80.65,83,40,5.75,scattered clouds
10,10,Vaini,TO,-21.2000,-175.2000,75.36,94,20,0.00,few clouds
12,12,Venice,US,27.0998,-82.4543,70.99,62,81,13.00,broken clouds
13,13,Kavieng,PG,-2.5744,150.7967,77.92,90,100,11.61,moderate rain
14,14,Jalu,LY,29.0331,21.5482,80.46,19,0,11.86,clear sky
16,16,Serowe,BW,-22.3833,26.7167,74.05,53,75,15.97,broken clouds


In [148]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                263
City                   263
Country                261
Lat                    263
Lng                    263
Max Temp               263
Humidity               263
Cloudiness             263
Wind Speed             263
Current Description    263
dtype: int64

In [149]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hilo,US,19.7297,-155.0900,69.53,86,100,5.75,light rain
5,5,Aksu,CN,41.1231,80.2644,76.42,13,11,5.37,few clouds
6,6,Sassandra,CI,4.9500,-6.0833,87.64,63,93,9.78,overcast clouds
7,7,Hithadhoo,MV,-0.6000,73.0833,83.62,64,57,7.94,broken clouds
9,9,Avarua,CK,-21.2078,-159.7750,80.65,83,40,5.75,scattered clouds


In [150]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,69.53,light rain,19.7297,-155.0900,
5,Aksu,CN,76.42,few clouds,41.1231,80.2644,
6,Sassandra,CI,87.64,overcast clouds,4.9500,-6.0833,
7,Hithadhoo,MV,83.62,broken clouds,-0.6000,73.0833,
9,Avarua,CK,80.65,scattered clouds,-21.2078,-159.7750,
10,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,
12,Venice,US,70.99,broken clouds,27.0998,-82.4543,
13,Kavieng,PG,77.92,moderate rain,-2.5744,150.7967,
14,Jalu,LY,80.46,clear sky,29.0331,21.5482,
16,Serowe,BW,74.05,broken clouds,-22.3833,26.7167,


In [151]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for indx, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    google_directions_api_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(google_directions_api_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[indx, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[indx, "Hotel Name"] = np.nan
        print("Hotel for city " + row["City"] + " not found, setting to nan and skipping.")
        

Hotel for city Adrar not found, setting to nan and skipping.
Hotel for city Manaquiri not found, setting to nan and skipping.
Hotel for city Luba not found, setting to nan and skipping.
Hotel for city Jutai not found, setting to nan and skipping.
Hotel for city Tirumullaivasal not found, setting to nan and skipping.
Hotel for city Sembakung not found, setting to nan and skipping.
Hotel for city Kilimatinde not found, setting to nan and skipping.
Hotel for city Kaniama not found, setting to nan and skipping.
Hotel for city Beloha not found, setting to nan and skipping.
Hotel for city Balaipungut not found, setting to nan and skipping.
Hotel for city Thinadhoo not found, setting to nan and skipping.
Hotel for city Morros not found, setting to nan and skipping.
Hotel for city Yumen not found, setting to nan and skipping.
Hotel for city Esparta not found, setting to nan and skipping.


In [152]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df = hotel_df.dropna()
hotel_clean_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,69.53,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
5,Aksu,CN,76.42,few clouds,41.1231,80.2644,Pudong Holiday Hotel
6,Sassandra,CI,87.64,overcast clouds,4.9500,-6.0833,la terrasse
7,Hithadhoo,MV,83.62,broken clouds,-0.6000,73.0833,Scoop Guest House
9,Avarua,CK,80.65,scattered clouds,-21.2078,-159.7750,Paradise Inn
10,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
12,Venice,US,70.99,broken clouds,27.0998,-82.4543,A Beach Retreat On Casey Key
13,Kavieng,PG,77.92,moderate rain,-2.5744,150.7967,Nusa Island Retreat
14,Jalu,LY,80.46,clear sky,29.0331,21.5482,صاحبة الفخامة
16,Serowe,BW,74.05,broken clouds,-22.3833,26.7167,Hill View Inn


In [153]:
# 8a. Create the output File (CSV)
output_data_file = os.path.join("WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [154]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
    <dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
    </dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [155]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,  info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))